<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-5---Discrete-time-Averaging-Systems" data-toc-modified-id="Chapter-5---Discrete-time-Averaging-Systems-1">Chapter 5 - Discrete-time Averaging Systems</a></span><ul class="toc-item"><li><span><a href="#Averaging-systems-achieving-consensus" data-toc-modified-id="Averaging-systems-achieving-consensus-1.1">Averaging systems achieving consensus</a></span></li><li><span><a href="#5.2-Averaging-system-reaching-asymptotic-disagreement" data-toc-modified-id="5.2-Averaging-system-reaching-asymptotic-disagreement-1.2">5.2 Averaging system reaching asymptotic disagreement</a></span></li><li><span><a href="#Apendix-5.5-Design-and-computation-of-centrality-measures" data-toc-modified-id="Apendix-5.5-Design-and-computation-of-centrality-measures-1.3">Apendix 5.5 Design and computation of centrality measures</a></span></li></ul></li></ul></div>

In [1]:
%matplotlib widget

# Import packages
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import sys, os

# For interactive graphs
import ipywidgets as widgets

# Import self defined functions
sys.path.insert(1, os.path.join(sys.path[0], '..'))  # Need to call this for importing library from parent folder
import lib  # General library

# Settings
custom_figsize= (6, 4) # Might need to change this value to fit the figures to your screen
custom_figsize_square = (5, 5) 

# Chapter 5 - Discrete-time Averaging Systems

These Jupyter Notebook scripts contain some examples, visualization and supplements accompanying the book "Lectures on Network Systems" by Francesco Bullo http://motion.me.ucsb.edu/book-lns/. These scripts are published with the MIT license. **Make sure to run the first cell above to import all necessary packages and functions and adapt settings in case.** In this script it is necessary to execute cell by cell chronologically due to reocurring examples (Tip: Use the shortcut Shift+Enter to execute each cell). Most of the functions are kept in separate files to keep this script neat.

## Averaging systems achieving consensus

In this section the three examples of the book are presented again to verify the results obtained and which lead as an introduction to Theorem 5.1 .

**First Example**

The first example is again the wireless network system from Chapter 1.2. First, we create some plots that include tha graph, binary matrix representation, spectrum and the network simulation. 

In [2]:
# Creating the Network System example as in Chapter 1.2 and as first example in chapter 5.1
G = nx.DiGraph()
G.add_edges_from([(0,1), (1,0), (0,0), (2,2), (1,2), (2,1), (2,3), (3,2), (1,3), (3,1), (3,3)])

# Define position of nodes in graph plot
pos = {0:[0.2,0.2],1:[.4,.2],2:[.4,.6],3:[.7,.6]}

# Define the adjacency matrix A
A = np.array([[0.5,0.5, 0., 0.],
              [1/4, 1/4, 1/4, 1/4],
              [0., 1/3, 1/3, 1/3],
              [0., 1/3, 1/3, 1/3]
])

fig, axs51 = plt.subplots(2, 2, figsize = (custom_figsize[0]*1.2, custom_figsize[1]*1.5))  # Init figure

# Draw network
nx.draw_networkx(G, pos, node_size=200, ax = axs51[0, 0], connectionstyle='arc3, rad = 0.1')
axs51[0, 0].margins(0.05) # Zooming out for better visualization

# Draw binary matrix representation
lib.plot_matrix_binary(A, axs51[1, 0])

# Draw spectrum
lib.plot_spectrum(A, axs51[0, 1]);

# Draw simulation results
x_0 = np.array([0.9, 0.3, 0.8, 0.6])
# Choosing the range of time step
t = 15
states = lib.simulate_network(A,x_0, t)  # Simulate network and save states for each time step in a t*n np.array
lib.plot_node_val_2D(states, x_0, t, axs51[1, 1])  # Visualize states in a 2D Graph
axs51[1, 1].set_title("Simulation of Network")
axs51[1,1].legend(loc=4, ncol=2)
fig.subplots_adjust(hspace=0.5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

As expected, we do not reach average consensus, since $A$ is not doubly-stochastic. However, we can calculate the consensus value by $w^T x(0)$, where $w$ is the left dominant eigenvector normed to 1 as demonstrated in the book.

In [3]:
# Left dominant eigenvectors
lambdas, eigv = np.linalg.eig(A.T)
left_dom = eigv[:, 0] / sum(eigv[:, 0])  # With Norm1 = 1

print("Left dominant eigenvector w:")
lib.matprint(left_dom)

print("\nFinal consensus value w^T x(0):")
print(left_dom.T @ x_0)

Left dominant eigenvector w:
|  0.166667  |
|  0.333333  |
|      0.25  |
|      0.25  |

Final consensus value w^T x(0):
0.6


**Second Example**

The second example is the robotic cyclic pursuit from section 1.6 of the book. Here we show, how average consensus is achieved.

In [ ]:
# Creating the Network System example as in Chapter 1.2 and as first example in chapter 5.1
G2 = nx.DiGraph()
G2.add_edges_from([(0,0), (0,1), (1,1), (1,2), (2,2), (2,3), (3,3), (3,4), (4,4), (4,5), (5,5), (5,0)])

# Define position of nodes in graph plot
pos2 = {0:[0.3,0.2],1:[.2,.5],2:[.3,.8],3:[.7,.8], 4:[.8,.5], 5:[0.7,0.2]}

# Define the adjacency matrix A
A2 = np.array([[1/2,1/2, 0., 0., 0., 0.],
               [0, 1/2, 1/2, 0., 0., 0.],
               [0., 0., 1/2, 1/2, 0., 0.],
               [0., 0., 0., 1/2, 1/2, 0.],
               [0., 0., 0., 0., 1/2, 1/2],
               [1/2, 0., 0., 0., 0., 1/2]
])

fig, axs512 = plt.subplots(2, 2, figsize = (custom_figsize[0]*1.2, custom_figsize[1]*1.5))  # Init figure

# Draw network
nx.draw_networkx(G2, pos2, node_size=200, ax = axs512[0, 0], connectionstyle='arc3, rad = 0.1')
axs512[0, 0].margins(0.05) # Zooming out for better visualization

# Draw binary matrix representation
lib.plot_matrix_binary(A2, axs512[1, 0])

# Draw spectrum
lib.plot_spectrum(A2, axs512[0, 1]);

# Draw simulation results
x_0_2 = np.array([0.1, 0.3, 0.4, 0.5, 0.6, 0.8])
# Choosing the range of time step
t = 50
states2 = lib.simulate_network(A2,x_0_2, t)  # Simulate network and save states for each time step in a t*n np.array
lib.plot_node_val_2D(states2, x_0_2, t, axs512[1, 1])  # Visualize states in a 2D Graph
axs512[1, 1].set_title("Simulation of Network")
axs512[1,1].legend(loc=4, ncol=3)
fig.subplots_adjust(hspace=0.5)

print("\nFinal average consensus value avg(x(0)):")
print(np.mean(x_0_2))

**Third Example**

In this example a not strongly connected matrix with its assciated digraph is shown. As mentioned in the book, we call such row-stochastic matrices indecomposable, since we still achieve consensus.

In [ ]:
# Creating the Network System example as in Chapter 1.2 and as first example in chapter 5.1
G3 = nx.DiGraph()
G3.add_edges_from([(0,1), (0,2), (1,4), (1,8), (2,4), (2,6), (2,7), (3,1), (3,6), (3,7), (4,1), (5,4), (6,2), (6,9), (7,0), (7,6), (8,0), (9,0), (9,2)])

# Define positon for nodes by layout
pos3 = nx.drawing.layout.spring_layout(G3)
# Extract Adjacency Matrix here and make it row stochastic
A3 = nx.linalg.graphmatrix.adjacency_matrix(G3, nodelist=range(0, G3.number_of_nodes())).toarray()
A3 = A3 / np.sum(A3, axis=1)[:, None]

fig, axs513 = plt.subplots(2, 2, figsize = (custom_figsize[0]*1.2, custom_figsize[1]*1.5))  # Init figure

# Draw network
nx.draw_networkx(G3, pos3, node_size=200, ax = axs513[0, 0], connectionstyle='arc3, rad = 0.1')
axs513[0, 0].margins(0.05) # Zooming out for better visualization

# Draw binary matrix representation
lib.plot_matrix_binary(A3, axs513[1, 0])

# Draw spectrum
lib.plot_spectrum(A3, axs513[0, 1]);

# Draw simulation results
x_0_3 = np.random.rand(10)
# Choosing the range of time step
t = 20
states3 = lib.simulate_network(A3,x_0_3, t)  # Simulate network and save states for each time step in a t*n np.array
lib.plot_node_val_2D(states3, x_0_3, t, axs513[1, 1])  # Visualize states in a 2D Graph
axs513[1, 1].set_title("Simulation of Network")
axs513[1,1].get_legend().remove()
fig.subplots_adjust(hspace=0.5)


## 5.2 Averaging system reaching asymptotic disagreement

In this section a minimum example for averaging systems reaching asymptotic disagreement is given. See in the plot, how different nodes reach different states, based on if the node belongs to a sink our which sink it senses. Below, it is even presented in an interactive environment.

In [ ]:
# Creating the Network System example as in Chapter 1.2 and as first example in chapter 5.1
G4 = nx.DiGraph()
G4.add_edges_from([(0,1), (0,3), (1,3), (1,2), (2,1), (2,3), (3,1), (3,2), (4,3), (4,5), (5,6), (6,5), (5,7), (7,6)])

# Define position of nodes in graph plot
pos4 = {0:[0.4,0.8],1:[.3,.5],2:[.4,.2],3:[.5,.5], 4:[.7,.8], 5:[0.7,0.5], 6:[0.8,0.4], 7:[0.7,0.3]}

# Extract Adjacency Matrix here and make it row stochastic
A4 = nx.linalg.graphmatrix.adjacency_matrix(G4, nodelist=range(0, G4.number_of_nodes())).toarray()
A4 = A4 / np.sum(A4, axis=1)[:, None]

fig, axs52 = plt.subplots(3, 1, figsize = (custom_figsize[0]*1.0, custom_figsize[1]*1.8))  # Init figure

# Draw network
nx.draw_networkx(G4, pos4, node_size=200, ax = axs52[0], connectionstyle='arc3, rad = 0.1')
axs52[0].margins(0.05) # Zooming out for better visualization

# Draw binary matrix representation
lib.plot_matrix_binary(A4, axs52[1])

# Draw spectrum
lib.plot_spectrum(A4, axs52[2]);

fig.subplots_adjust(hspace=0.5)



fig, ax521 = plt.subplots(figsize = (custom_figsize[0]*1.2, custom_figsize[1]*1.2))  # Init figure

# Draw simulation results
x_0_4 = np.random.rand(8)
# Choosing the range of time step
t4 = 20
states4 = lib.simulate_network(A4,x_0_4, t4)  # Simulate network and save states for each time step in a t*n np.array
lib.plot_node_val_2D(states4, x_0_4, t4, ax521)  # Visualize states in a 2D Graph
ax521.set_title("Simulation of Network")
ax521.legend(loc=4, ncol=3);


In [ ]:
fig, ax523 = plt.subplots(figsize=custom_figsize)
fig, v_bound, pos4 = lib.init_network_sim_plot(G4, states4, fig, pos=pos4)

def inter(timestep):
    lib.update_network(timestep['new'], G=G4, states_m=states4, ax=ax523, vbound=v_bound, pos=pos4)
    #ax3.margins(0.20) # Zooming out for better visualization
    return None

# Plot initial configuration
lib.update_network(0, G=G4, states_m=states4, ax=ax523, vbound=v_bound, pos=pos4)


# Widget
# If this cell is executed twice we are making sure in the following, that the previous widget instances are all closed
try:
    [c.close() for c in widget52.children]  # Note: close_all() does also affect plot, thus list compr.
except NameError:  # Only want to except not defined variable error
    pass

widget52 = lib.create_widgets_play_slider(fnc=inter, minv=0, maxv=t4-1, step=1, play_speed=1000)

display(widget52)

## Apendix 5.5 Design and computation of centrality measures

The results from 5.13 are visualized again here:

In [ ]:
# define graph
G55 = nx.Graph(); 
G55.add_nodes_from(range(0,10));  
G55.add_edges_from([(0,1), (1,5), (1,6), (1,2), (2,3), (2,7), (6,7), (7,8), (3,8), (8,9), (9,4), (9,10)])
pos55 = {0:[0.1,0.8],1:[.2,.8],2:[.3,.8],3:[.4,.8], 4:[.5,.8], 5:[0.1,0.4], 6:[0.2,0.4], 7:[0.3,0.4], 8:[0.4,0.4], 9:[0.5,0.4], 10:[0.6,0.4]}

# Node 2 has the highest degree centrality
degree_centrality = nx.degree_centrality(G55)

# Node 3 has the highest eigenvector centrality
eigenvector_centrality = nx.eigenvector_centrality_numpy(G55)

# Node 8 has highest closeness centrality
closeness_centrality = nx.closeness_centrality(G55)

# Node 9 has highest closeness centrality
betweenness_centrality = nx.betweenness_centrality(G55)

In [ ]:
deg_cen = np.array(list(degree_centrality.values()))[None, :]
eig_cen = np.array(list(eigenvector_centrality.values()))[None, :]
clo_cen = np.array(list(closeness_centrality.values()))[None, :]
bet_cen = np.array(list(betweenness_centrality.values()))[None, :]

all_cen = np.vstack((deg_cen, eig_cen, clo_cen, bet_cen))

lib.matprint(all_cen)

titles = ["Degree Centrality", "Eigenvector Centrality", "Closeness Centrality", "Betweenness Centrality"]

In [ ]:
for i in range(len(titles)):
    fig, ax55 = plt.subplots(figsize=custom_figsize)
    fig, v_bound, pos55 = lib.init_network_sim_plot(G55, all_cen[i, :][None, :], fig, pos=pos55)
    lib.update_network(0, G55, all_cen[i, :][None, :], ax55, v_bound, pos55, labels=False)
    ax55.set_title(titles[i])